Замена view-функций на CBV заметно облегчила код, но при рефакторинге приложение утратило важный сервис — теперь нигде не отображаются результаты подсчёта времени, оставшегося до дня рождения. 

Создадим страницу, где будет отображаться отдельная запись; на этой же странице будем отображать и количество дней, оставшихся до дня рождения.

В Django есть специальный view-класс для отображения отдельных объектов: `DetailView`. Унаследуем от него собственный класс — `BirthdayDetailView`; отображать отдельные записи будем на страницах с адресом вида *birthday/<pk>/*.

Начнём с маршрутизатора:

```py
# birthday/urls.py
from django.urls import path

from . import views

app_name = 'birthday'

urlpatterns = [
    path('', views.BirthdayCreateView.as_view(), name='create'),
    path('list/', views.BirthdayListView.as_view(), name='list'),
    path('<int:pk>/', views.BirthdayDetailView.as_view(), name='detail'),
    path('<int:pk>/edit/', views.BirthdayUpdateView.as_view(), name='edit'),
    path('<int:pk>/delete/', views.BirthdayDeleteView.as_view(), name='delete'),
] 
```

***
## DetailView

Теперь очередь шаблона и CBV.

Опишем представление для отдельного объекта; для начала достаточно объявить класс и в нём указать модель.

```py
# birthday/views.py
...
from django.views.generic import (
    CreateView, DeleteView, DetailView, ListView, UpdateView
)
...


class BirthdayDetailView(DetailView):
    model = Birthday
```

Теперь шаблон. Для начала выясним, какое имя шаблона ожидает увидеть класс `DetailView`. Проще всего будет заглянуть на [сайт, посвящённый CBV](https://docs.djangoproject.com/en/5.1/topics/class-based-views/), — там есть вся необходимая информация:

```py
template_name_suffix = '_detail' 
```

Создайте шаблон с суффиксом *_detail* — он будет служить для отображения отдельного объекта модели `Birthday`:

```html
<!-- birthday/birthday_detail.html -->
{% extends "base.html" %}

{% block content %}
  ID записи: {{ object.id }}
  <hr>
  {% if birthday.image %}
    <div>
      <!-- Картинку сделаем побольше, чем на странице list/: высотой 200px -->
      <img src="{{ birthday.image.url }}" height="200">
    </div>
  {% endif %}
   <h2>Привет, {{ object.first_name }} {{ object.last_name }}</h2>      
  {% if birthday_countdown == 0 %}
    <p>С днём рождения!</p>
  {% else %}
    <p>Осталось дней до дня рождения: {{ birthday_countdown }}!</p>
  {% endif %}
{% endblock content %} 
```

***
## Переопределение словаря контекста в DetailView

Чтобы дополнить или переопределить словарь контекста — применяют метод `get_context_data()`. 

Переопределим этот метод, а в нём

* Получим словарь с контекстом из родительского метода `get_context_data()`:

```py
  context = super().get_context_data(**kwargs)
```
   

* Дополним словарь новым ключом; значением этого ключа будет вызов функции `calculate_birthday_countdown()`:

```py
  context['birthday_countdown'] = calculate_birthday_countdown(...)
```
   

* В качестве аргумента в функцию нужно передать дату рождения — значение поля `birthday` объекта модели. Сам объект доступен в атрибуте `self.object`:

```py
  context['birthday_countdown'] = calculate_birthday_countdown(
      self.object.birthday
  )
```

Чтобы пользователь мог попасть на отдельную страницу записи — добавьте в шаблон *birthday_list.html* ссылку на страницу объекта, например, вот так:

```py
...
<div class="col-10">  
  <div>
    {{ birthday.first_name }} {{ birthday.last_name }} - {{ birthday.birthday }}<br>
    <a href="{% url 'birthday:detail' birthday.id %}">Сколько до дня рождения?</a>
  </div>
... 
```

***
## Редирект на страницу отдельной записи

После создания или редактирования записи пользователь перенаправляется на общий список записей, но это неудобно: чтобы убедиться, что запись создана или изменена, пользователю придётся отыскать её в списке — а список может оказаться довольно велик.

Но теперь в приложении появилась отдельная страница записи — и после создания или редактирования записи будет логично переадресовывать пользователя именно на неё.

Настроить переадресацию через атрибут CBV `success_url` не получится, ведь для этого надо динамически подставить в адрес id страницы. Есть два основных способа, которыми можно решить эту задачу: 

1. Можно описать в CBV метод `get_success_url()`, который будет формировать нужную ссылку. Недостаток у этого метода в том, что во избежание дублирования кода придётся создать отдельный миксин и «подмешивать» его во все нужные CBV.

2. Другой вариант — описать в модели, с которой работают CBV, метод `get_absolute_url()` — «получить абсолютный URL объекта». Тогда, если в CBV не указаны атрибут `success_url` или метод `get_success_url()`, которые обладают бо́льшим приоритетом, CBV будет обращаться именно к методу `get_absolute_url()`. И никаких дополнительных миксинов добавлять не потребуется. Именно такой способ предлагает использовать [документация](https://docs.djangoproject.com/en/3.2/topics/class-based-views/generic-editing/#model-forms).

Добавьте метод `get_absolute_url()` в модель `Birthday` в файле *birthday/models.py*.

```py
# birthday/models.py
...
# Импортируем функцию reverse() для получения ссылки на объект.
from django.urls import reverse
...


class Birthday(models.Model):
    ...
    
    class Meta:
        ...

    def get_absolute_url(self):
        # С помощью функции reverse() возвращаем URL объекта.
        return reverse('birthday:detail', kwargs={'pk': self.pk}) 
```

Теперь можно убрать из классов `BirthdayCreateView` и `BirthdayUpdateView` атрибут `success_url`; после отправки формы пользователь будет переадресован на страницу конкретной записи. Значит, и миксин `BirthdayMixin` с атрибутом `success_url` тоже не особо нужен, его можно убрать. 

Файл *views.py* получится таким:

```py
# birthday/views.py
from django.views.generic import (
    CreateView, DeleteView, DetailView, ListView, UpdateView
)
from django.urls import reverse_lazy

from .forms import BirthdayForm
from .models import Birthday
from .utils import calculate_birthday_countdown

class BirthdayListView(ListView):
    model = Birthday
    ordering = 'id'
    paginate_by = 10

class BirthdayCreateView(CreateView):
    model = Birthday
    form_class = BirthdayForm

class BirthdayUpdateView(UpdateView):
    model = Birthday
    form_class = BirthdayForm

class BirthdayDeleteView(DeleteView):
    model = Birthday
    success_url = reverse_lazy('birthday:list')

class BirthdayDetailView(DetailView):
    model = Birthday

    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['birthday_countdown'] = calculate_birthday_countdown(
            self.object.birthday
        )
        return context 
```

Полный список Class-Based Views, встроенных в Django, можно посмотреть [в документации](https://docs.djangoproject.com/en/3.2/ref/class-based-views/). Помимо этого, есть [специальный сайт](https://ccbv.co.uk/projects/Django/3.2/), посвящённый Django CBV. Там удобная структура, перечислены все методы, атрибуты, родительские классы и потомки CBV и выложено много другой информации. Этот сайт рекомендован и в [документации Django](https://docs.djangoproject.com/en/3.2/ref/class-based-views/flattened-index/).